In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from HeteroscedasticGP.Models import BasicRegressor

# Make data

In [ ]:
from statsmodels.datasets import get_rdataset

# Load the motorcycle data from R's 'MASS' package
dataset = get_rdataset("mcycle", "MASS").data

# Extract features (time) and target (acceleration)
X = dataset["times"].values.reshape(-1, 1)  # 2D array
y = dataset["accel"].values  # 1D array

In [ ]:
plt.figure(figsize=(8, 4))
plt.scatter(X, y, c="black", label="Observations")
plt.legend()
plt.title("Motorcycle Data")
plt.grid()
plt.show()

# Fit model

In [ ]:
y_mean = np.mean(y)
y_std = np.std(y)
y_standardised = (y - y_mean) / y_std

X_mean = np.mean(X)
X_std = np.std(X)
X_standardised = (X - X_mean) / X_std

m = BasicRegressor(ARD=False)
z0 = np.log(np.random.rand(len(y)))
f_params0 = {'lengthscale': 1}
z_params0 = {'lengthscale': 1}
m.train(X=X_standardised, y=y_standardised, f_params0=f_params0, z_params0=z_params0, z0=z0)

In [ ]:
print(m.f_params_opt)
print(m.z_params_opt)

# Predictions

In [ ]:
fig, ax = plt.subplots()
ax.plot(X, np.sqrt(np.exp(m.z_opt)), marker='.')
ax.set_ylabel('Estimated noise std')
ax.grid()

In [ ]:
mu_star, var_star = m.predict(X_standardised)

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(X, y_standardised, c="black", label="Observations")
plt.plot(X, mu_star, "red", lw=2, label="Mean prediction")
plt.fill_between(
    X.ravel(),
    mu_star - 3 * np.sqrt(var_star),
    mu_star + 3 * np.sqrt(var_star),
    color="red", alpha=0.2, label="Predicted ±3σ"
)
plt.legend()
plt.title("Basic Regression Model Applied to Motorcycle Data")
plt.grid()
plt.show()